# Aula 10

# Trabalhando com o PySpark


### Introdução


Usaremos o conjunto de dados relacionado a campanhas de marketing direto (chamadas telefônicas) de uma instituição bancária. O objetivo da classificação é prever se o cliente irá realizar (Sim/Não) um depósito a prazo. O conjunto de dados pode ser baixado no link [aqui](https://archive.ics.uci.edu/dataset/222/bank+marketing).

In [19]:
import gdown
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession

# URL do Google Drive
url = 'https://drive.google.com/uc?id=19M95VTZyZ5HzBxlIALMr1Qyo5TbMc1Jy'

# Baixando o arquivo
output = 'file.csv'
gdown.download(url, output, quiet=False)

# Inicializando o SparkSession
spark = SparkSession.builder.appName('ml-bank').getOrCreate()

# Lendo o arquivo CSV baixado
df = spark.read.csv(output, header=True, inferSchema=True)
df.printSchema()

Downloading...
From: https://drive.google.com/uc?id=19M95VTZyZ5HzBxlIALMr1Qyo5TbMc1Jy
To: e:\Senac\Aulas\CursoBigData\Semana10\file.csv
100%|██████████| 919k/919k [00:00<00:00, 2.34MB/s]


root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: integer (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- deposit: string (nullable = true)



Mas podemos converte em um DataFrame do PySpark para um DataFrame do Pandas. O método `df.take(5)` obtém as primeiras 5 linhas do DataFrame do PySpark `df`, e a função `pd.DataFrame(..., columns=df.columns)` converte essas 5 linhas em um DataFrame do Pandas, preservando os nomes das colunas originais do DataFrame do PySpark. O resultado é um DataFrame do Pandas contendo as primeiras 5 linhas do DataFrame original do PySpark, facilitando assim a visualização e análise rápida dos dados em um ambiente local utilizando as funcionalidades do Pandas.

O código a seguir agrupa os dados pela coluna `'deposit'`, contabilizando quantos clientes realizaram ('Yes') e quantos não realizaram ('No') um depósito a prazo. Em seguida, converte o resultado para um DataFrame do Pandas, facilitando a manipulação e visualização dos dados. O resultado final é uma tabela que mostra a contagem de registros para cada valor na coluna `'deposit'`.

Primeiro, vamos uma lista chamada `numeric_features`, que contém os nomes das colunas do DataFrame `df` que têm o tipo de dado inteiro. Em seguida, selecionamos essas colunas do DataFrame e aplicamos o método `describe()` para obter estatísticas descritivas, como média, desvio padrão, valores mínimo e máximo. Por fim, convertemos o resultado para um DataFrame do Pandas e transpomos a tabela para facilitar a visualização dessas estatísticas.

A principal conclusão da matriz de correlação apresentada é que a maioria das variáveis numéricas não possuem fortes correlações entre si. No entanto, há uma correlação moderadamente forte entre as variáveis `pdays` e `previous` (0.507272). Isso indica que há uma relação significativa entre o número de dias desde que um cliente foi contatado por uma campanha anterior (`pdays`) e o número de contatos realizados antes dessa campanha (`previous`). Em geral, as outras correlações são bastante fracas, sugerindo que as variáveis numéricas selecionadas são relativamente independentes umas das outras no contexto deste conjunto de dados.

Gerando a matriz de dispersão:

### Usando os dados para fazer previsão de cenários futuros

Vamos realizar a indexação e codificação de colunas categóricas de um DataFrame do PySpark. Primeiro, vamos importar as bibliotecas necessárias e define uma lista de colunas categóricas. Para cada coluna, vamos criar um `StringIndexer` que converte os valores categóricos em índices numéricos. Em seguida, usa um `OneHotEncoderEstimator` para transformar esses índices em vetores binários one-hot. Esses transformadores são armazenados em uma lista chamada `stages`, que será usada em um pipeline para aplicar essas transformações aos dados. O resultado final é um DataFrame com as colunas categóricas convertidas em vetores binários, facilitando a análise e o uso em algoritmos de aprendizado de máquina.

In [31]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

categoricalColumns = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'poutcome']

stages = []

for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol + 'Index')
    encoder = OneHotEncoder(inputCol=stringIndexer.getOutputCol(), outputCol=categoricalCol + 'classVec')
    stages += [stringIndexer, encoder]

# Visualização do estágio
for stage in stages:
    print(stage)

StringIndexer_542057a63ce1
OneHotEncoder_29d0321f31bf
StringIndexer_d0ee5d93867a
OneHotEncoder_235f6c8f55b0
StringIndexer_110e000c7105
OneHotEncoder_d6d417d6ffae
StringIndexer_8dd09672383f
OneHotEncoder_b6fe1db8d2a4
StringIndexer_9896f3b6c173
OneHotEncoder_b491d0309705
StringIndexer_3b3f56df9f05
OneHotEncoder_c2ba1dce4035
StringIndexer_4e94cffbf0ef
OneHotEncoder_ae1435ab4f8c
StringIndexer_df86b9d65d00
OneHotEncoder_d69607603f0c


Usamos o StringIndexer novamente para codificar nossas etiquetas em índices de etiquetas. Em seguida, usamos o VectorAssembler para combinar todas as colunas de características em uma única coluna de vetor.

Pipeline

Usamos o Pipeline para encadear múltiplos Transformadores e Estimadores juntos para especificar nosso fluxo de trabalho de aprendizado de máquina. Os estágios de um Pipeline são especificados como um array ordenado.

Separando o conjunto em treino e teste.

### Criando um Modelo de Regressão Logística

Em um modelo de regressão logística, os gráficos "Beta Coefficients", "ROC Curve" e "Precision x Recall" servem para diferentes propósitos de análise e interpretação do desempenho do modelo:

1. **Beta Coefficients**:
   - **Propósito**: Este gráfico mostra os coeficientes estimados para cada variável independente (ou característica) no modelo de regressão logística.
   - **Interpretação**: Os coeficientes indicam a direção e a magnitude da associação entre as variáveis independentes e a variável dependente. Coeficientes positivos indicam que o aumento na variável independente está associado a um aumento na probabilidade do resultado, enquanto coeficientes negativos indicam o oposto.

2. **ROC Curve (Receiver Operating Characteristic Curve)**:
   - **Propósito**: Este gráfico é usado para avaliar a capacidade do modelo em distinguir entre as classes (por exemplo, positivo vs. negativo).
   - **Interpretação**: A curva ROC plota a taxa de verdadeiros positivos (sensibilidade) contra a taxa de falsos positivos (1 - especificidade) para diferentes limiares de classificação. A área sob a curva ROC (AUC) é uma medida do desempenho do modelo; quanto mais próximo de 1, melhor a capacidade de discriminação do modelo.

3. **Precision x Recall**:
   - **Propósito**: Este gráfico é usado para avaliar o equilíbrio entre a precisão (quantos dos exemplos classificados como positivos são realmente positivos) e a recall (quantos dos exemplos realmente positivos são classificados corretamente pelo modelo).
   - **Interpretação**: A curva de Precisão x Recall é especialmente útil em cenários de classes desbalanceadas, onde uma classe é muito mais frequente do que a outra. A análise desse gráfico ajuda a entender o trade-off entre precisão e recall e a escolher o limiar de decisão mais adequado para o problema em questão.

Em resumo:
- **Beta Coefficients**: Indicam a influência das variáveis independentes no modelo.
- **ROC Curve**: Avalia a capacidade do modelo de distinguir entre classes.
- **Precision x Recall**: Avalia o equilíbrio entre precisão e recall, especialmente útil para classes desbalanceadas.